In [1]:
import sqlite3

In [2]:
import pandas as pd

In [3]:
from datetime import datetime

In [4]:
import matplotlib.pyplot as plt

In [5]:
plt.style.use('fivethirtyeight')
%matplotlib inline

In [8]:
# find the chat.db file
conn = sqlite3.connect('chat.db')
cur = conn.cursor()

# query the database to get all the table names
cur.execute(" select name from sqlite_master where type = 'table' ")

for name in cur.fetchall():
    print(name)

('_SqliteDatabaseProperties',)
('deleted_messages',)
('sqlite_sequence',)
('chat_handle_join',)
('chat_message_join',)
('message_attachment_join',)
('handle',)
('message',)
('chat',)
('attachment',)
('sync_deleted_messages',)
('message_processing_task',)
('sync_deleted_chats',)
('sync_deleted_attachments',)
('kvtable',)
('sqlite_stat1',)


In [9]:
# create panda dataframe
messages = pd.read_sql_query('''select *, datetime(date/1000000000 + strftime("%s", "2001-01-01") ,"unixepoch","localtime")  as date_utc from message''', conn) 

handles = pd.read_sql_query("select * from handle", conn)
chat_message_joins = pd.read_sql_query("select * from chat_message_join", conn)

In [10]:
messages.rename(columns={'ROWID' : 'message_id'}, inplace = True)

In [11]:
# rename appropriately the handle and apple_id/phone_number as well
handles.rename(columns={'id' : 'phone_number', 'ROWID': 'handle_id'}, inplace = True)

In [13]:
# these fields are only for ease of datetime analysis (e.g., number of messages per month or year)
messages['message_date'] = messages['date']
messages['timestamp'] = messages['date_utc'].apply(lambda x: pd.Timestamp(x))
messages['date'] = messages['timestamp'].apply(lambda x: x.date())
messages['month'] = messages['timestamp'].apply(lambda x: int(x.month))
messages['year'] = messages['timestamp'].apply(lambda x: int(x.year))

In [14]:
# merge the messages with the handles
merge_level_1 = pd.merge(messages[['text', 'handle_id', 'date','message_date' ,'timestamp', 'month','year','is_sent', 'message_id']],  handles[['handle_id', 'phone_number']], on ='handle_id', how='left')

# and then that table with the chats
df_messages = pd.merge(merge_level_1, chat_message_joins[['chat_id', 'message_id']], on = 'message_id', how='left')


print(len(df_messages))

91616


In [15]:
print(df_messages.head())

                                    text  handle_id        date  \
0                Do you wanna get lunch?          1  2017-04-22   
1  is there a chinese name for drmartens          1  2017-04-25   
2                   What’s dramartens...          1  2017-04-25   
3              dr martens... the shoes 🙃          1  2017-04-25   
4               Lol why there’s no space          1  2017-04-25   

         message_date           timestamp  month  year  is_sent  message_id  \
0  514607241000000000 2017-04-22 22:27:21      4  2017        1           1   
1  514809038000000000 2017-04-25 06:30:38      4  2017        0           2   
2  514809062000000000 2017-04-25 06:31:02      4  2017        1           3   
3  514809107000000000 2017-04-25 06:31:47      4  2017        0           4   
4  514809124000000000 2017-04-25 06:32:04      4  2017        1           5   

                  phone_number  chat_id  
0  alecdortkardesler@gmail.com      1.0  
1  alecdortkardesler@gmail.com      1.

In [16]:
# save the combined table for ease of read for future analysis!
df_messages.to_csv('./imessages_high_sierra.csv', index = False, encoding='utf-8')

In [17]:
import csv
import python

ModuleNotFoundError: No module named 'python'

In [18]:
import json

In [37]:
# Function to convert a CSV to JSON 
# Takes the file paths as arguments 
def make_json(csvFilePath, jsonFilePath): 
      
    # create a dictionary 
    data = {} 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in csvReader: 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows['message_id'] 
            data[key] = rows 
  
    # Open a json writer, and use the json.dumps()  
    # function to dump data 
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonf.write(json.dumps(data, indent=4)) 

In [38]:
csvFilePath = r'./avery-zeyao.csv'
jsonFilePath = r'./avery-zeyao.json'

In [39]:
make_json(csvFilePath, jsonFilePath)